In [288]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython import display
import keras
from keras.layers import Dense, Activation
from keras.models import Sequential
import tensorflow as tf

In [289]:
import gym

env = gym.make("FrozenLake8x8-v0")
env.reset()

#plt.imshow(env.render('rgb_array'))
print("Observation space:", env.observation_space)
print("Action space:", env.action_space)
print('observation space:', env.observation_space)
print('action space:', env.action_space)
print("taking action")
new_obs, reward, is_done, _ = env.step(1)

print("new observation code:", new_obs)
print("reward:", reward)
print("is game over?:", is_done)


Observation space: Discrete(64)
Action space: Discrete(4)
observation space: Discrete(64)
action space: Discrete(4)
taking action
new observation code: 0
reward: 0.0
is game over?: False


In [290]:
n_states = env.observation_space.n
n_actions = env.action_space.n
print("Number of States: ", n_states)
print("Number of Actions: ", n_actions)

Number of States:  64
Number of Actions:  4


In [291]:
#Input policy to be evaluated

policy = np.ones((n_states , n_actions))/n_actions
print("Shape of Policy Matrix" , policy.shape)

#Initialize array V(s) = 0

V = np.zeros(n_states)
discount_factor = 1.0
theta = 0.00001

Shape of Policy Matrix (64, 4)


In [292]:
env.P[3][3]

[(0.3333333333333333, 4, 0.0, False),
 (0.3333333333333333, 3, 0.0, False),
 (0.3333333333333333, 2, 0.0, False)]

### Policy Evaluation

In [298]:
def policy_evaluation(policy, env, discount_factor = 1.0 , theta = 0.00001 ):
    #Repeat:
    while True:
        delta = 0.0        #delta <- 0
    
        for s in range(n_states):   #For each state:
            v = 0
            c = 0
            for a , policy_prob in enumerate(policy[s]):                            #Expression---
                for prob, next_state, reward, done  in env.P[s][a]:
                    v += policy_prob * prob *(reward + discount_factor*V[next_state])     # V(s) = sum(policy(a|s) * sum(prob(s', r|s,a)) * (reward + gamma*V(s'))                                                       #---
            delta = max(delta , abs(v - V[s]))            #delta <- max(delta , |v - V[s]|)
            V[s] = v

        if(delta<theta):                    #terminate on delta < theta (theta is a small positive number)
            break
    return V
        # Output V ~= vpi

In [299]:
vpi = policy_evaluation(policy , env) 

In [300]:
vpi

array([1.77291852e-03, 2.05231760e-03, 2.71200668e-03, 4.02682148e-03,
       6.45060006e-03, 9.69730771e-03, 1.33323211e-02, 1.58487758e-02,
       1.52729353e-03, 1.69427642e-03, 2.07670686e-03, 2.93669579e-03,
       5.64863740e-03, 9.33260477e-03, 1.44768170e-02, 1.83922528e-02,
       1.13470573e-03, 1.13113238e-03, 9.71534634e-04, 0.00000000e+00,
       3.88354185e-03, 7.51853274e-03, 1.68629320e-02, 2.48647709e-02,
       7.60085659e-04, 7.30250376e-04, 6.80298040e-04, 7.62713639e-04,
       2.37149567e-03, 0.00000000e+00, 2.05998575e-02, 3.93483800e-02,
       4.24537163e-04, 3.53058351e-04, 2.58339098e-04, 0.00000000e+00,
       4.84014067e-03, 1.15878559e-02, 2.61929417e-02, 7.25859868e-02,
       1.65174830e-04, 0.00000000e+00, 0.00000000e+00, 1.44772835e-03,
       5.40187309e-03, 1.53198027e-02, 0.00000000e+00, 1.52219331e-01,
       7.23056825e-05, 0.00000000e+00, 1.09152955e-04, 3.89220327e-04,
       0.00000000e+00, 4.42895328e-02, 0.00000000e+00, 3.84073024e-01,
      